[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [ ]:
import torch
import pandas as pd

## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [ ]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# D_in, H1, H2, D_out = 784, 100, 50, 10
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H1),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H1, H2),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H2, D_out),
# )


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [ ]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [ ]:
print(outputs[0][:])

tensor([-0.0394, -0.0911, -0.1514,  0.1468,  0.2993,  0.0721, -0.1777,  0.0992,
         0.2579, -0.1849], grad_fn=<SliceBackward0>)


Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [ ]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [ ]:
# Reemplaza 'dataset.csv' con la ruta correcta a tu archivo CSV
data = pd.read_csv('TMNIST_Data.csv')

# Eliminar todas las columnas con valores NaN
data = data.dropna(axis=1)

# Llenar los valores faltantes con un valor específico, como cero (0)
data = data.fillna(0)

Y = data['labels']
X = data.drop(columns=['labels', 'names'])

X = X.apply(pd.to_numeric, errors='coerce')
X = X.fillna(0)

X.shape, Y.shape

((29900, 784), (29900,))

In [ ]:
import numpy as np

# normalización y split
import numpy as np
x_2=np.array(X)
y_2=np.array(Y)

# normalización y split

X_train =x_2[:19900] / 255.
X_test =x_2[10000:] / 255.
y_train = y_2[:19900].astype(int)
y_test = y_2[10000:].astype(int)



# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.float32), Y[60000:].astype(np.float32)

In [ ]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
torch.cuda.is_available()

True

In [ ]:
print(X)

       1  2  3  4  5  6  7  8  9  10  ...  775  776  777  778  779  780  781  \
0      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
1      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
2      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
3      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
4      0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
...   .. .. .. .. .. .. .. .. ..  ..  ...  ...  ...  ...  ...  ...  ...  ...   
29895  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
29896  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
29897  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
29898  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   
29899  0  0  0  0  0  0  0  0  0   0  ...    0    0    0    0    0    0    0   

       782  783  784  
0        0    0 

In [ ]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 100
lr = 0.8
log_each = 10
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 10/100 Loss 1.69799
Epoch 20/100 Loss 1.31352
Epoch 30/100 Loss 0.99070
Epoch 40/100 Loss 0.81867
Epoch 50/100 Loss 0.69483
Epoch 60/100 Loss 0.60898
Epoch 70/100 Loss 0.54577
Epoch 80/100 Loss 0.49711
Epoch 90/100 Loss 0.45836
Epoch 100/100 Loss 0.42670


Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.9603015075376884

Existen algunos tipos de capas que se comportan diferente en función de si estamos entrenando la red o usándola para generar predicciones. Podemos controlar el modo en el que queremos que esté nuestra red con las funciones `train` y `eval`.

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente.

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.68165
Epoch 20/100 Loss 1.24153
Epoch 30/100 Loss 0.94897
Epoch 40/100 Loss 0.76777
Epoch 50/100 Loss 0.65363
Epoch 60/100 Loss 0.57462
Epoch 70/100 Loss 0.51638
Epoch 80/100 Loss 0.47147
Epoch 90/100 Loss 0.43565
Epoch 100/100 Loss 0.40634


0.9594472361809046

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [ ]:
# creamos una clase que hereda de `torch.nn.Module`

class ModeloPersonalizado(torch.nn.Module):

    # constructor
    def __init__(self, D_in, H, D_out):

        # llamamos al constructor de la clase madre
        super(ModeloPersonalizado, self).__init__()

        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones:

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [ ]:
model = ModeloPersonalizado(784, 100, 10)
# Codigo para saber si el modelo esta votando los datos en las cantidades correctas
x_prueba = torch.randn(64, 784)
print(x_prueba)
outputs = model(x_prueba)
outputs.shape

tensor([[-0.6461, -0.0207, -1.3515,  ..., -1.6513,  0.4883,  1.1908],
        [ 0.0965, -0.0683, -0.6054,  ..., -0.7764,  0.0158,  0.3123],
        [ 0.4723, -1.3714, -0.8379,  ..., -0.1985, -0.6113, -0.3834],
        ...,
        [-1.0806,  1.1132,  0.4798,  ..., -0.7306, -2.0824,  0.3731],
        [-0.0225,  0.3533,  1.4645,  ...,  0.7638, -0.1237,  0.7456],
        [-1.3046,  1.4031, -0.3966,  ..., -0.2055,  1.0389,  0.5899]])


torch.Size([64, 10])

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [ ]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.84291
Epoch 20/100 Loss 1.31924
Epoch 30/100 Loss 1.01732
Epoch 40/100 Loss 0.84021
Epoch 50/100 Loss 0.71162
Epoch 60/100 Loss 0.62277
Epoch 70/100 Loss 0.55752
Epoch 80/100 Loss 0.50740
Epoch 90/100 Loss 0.46757
Epoch 100/100 Loss 0.43507


0.9596984924623115

Aquí puedes ver otro ejemplo de como definir un `MLP` con conexiones residuales, algo que no podemos hacer simplemente usando un modelo secuencial.

In [ ]:
class ModelCustom2(torch.nn.Module):

    def __init__(self, D_in, H, D_out):
        super(ModelCustom2, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [ ]:
model = ModelCustom2(784, 100, 10).to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.35250
Epoch 20/100 Loss 0.95013
Epoch 30/100 Loss 0.74412
Epoch 40/100 Loss 0.62687
Epoch 50/100 Loss 0.55023
Epoch 60/100 Loss 0.49566
Epoch 70/100 Loss 0.45452
Epoch 80/100 Loss 0.42220
Epoch 90/100 Loss 0.39604
Epoch 100/100 Loss 0.37434


0.9515075376884422

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Accediendo a las capas de una red

En ocasiones queremos acceder a una capa en particular de nuestra red. Para ello, podemos acceder utilizando su nombre.

In [ ]:
model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [ ]:
model.fc1

Linear(in_features=784, out_features=100, bias=True)

También podemos acceder directamente a los tensores que contienen los parámetros con las propiedades adecuadas

In [ ]:
model.fc1.weight

Parameter containing:
tensor([[-0.0286, -0.0087, -0.0101,  ..., -0.0174,  0.0307, -0.0234],
        [ 0.0066, -0.0148, -0.0253,  ...,  0.0345,  0.0058, -0.0169],
        [ 0.0014, -0.0051, -0.0250,  ..., -0.0078, -0.0123,  0.0313],
        ...,
        [-0.0258, -0.0084, -0.0265,  ..., -0.0222,  0.0007,  0.0324],
        [ 0.0286,  0.0339,  0.0326,  ...,  0.0030,  0.0079,  0.0109],
        [ 0.0189, -0.0294, -0.0232,  ...,  0.0259, -0.0292,  0.0260]],
       device='cuda:0', requires_grad=True)

In [ ]:
model.fc1.bias

Parameter containing:
tensor([-0.0071,  0.0502, -0.0033,  0.0208,  0.0371,  0.0128,  0.0455,  0.0254,
         0.0317,  0.0383,  0.0115,  0.0599, -0.0245,  0.0411,  0.0477,  0.0390,
         0.0577,  0.0183,  0.0105,  0.0203,  0.0380,  0.0112,  0.0342, -0.0099,
        -0.0123,  0.0127,  0.0487, -0.0154,  0.0283,  0.0413,  0.0152,  0.0119,
        -0.0193,  0.0022, -0.0152,  0.0622, -0.0067,  0.0446,  0.0093, -0.0058,
         0.0135,  0.0490,  0.0496,  0.0381,  0.0668,  0.0438, -0.0028,  0.0509,
         0.0320,  0.0117, -0.0094,  0.0154, -0.0131,  0.0061, -0.0169,  0.0047,
         0.0341, -0.0223, -0.0017,  0.0573,  0.0126,  0.0202, -0.0029,  0.0383,
         0.0053,  0.0141,  0.0410,  0.0108,  0.0416,  0.0181, -0.0150,  0.0020,
         0.0080,  0.0554, -0.0065,  0.0205,  0.0231,  0.0603,  0.0148,  0.0371,
         0.0340,  0.0052, -0.0035,  0.0073,  0.0277, -0.0382,  0.0074,  0.0287,
         0.0199, -0.0172,  0.0285,  0.0188,  0.0231,  0.0522,  0.0476,  0.0400,
        -0.0088, -

Es posible sobreescribir una capa de la siguiente manera

In [ ]:
model.fc2 = torch.nn.Linear(100, 1)

model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=100, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

Ahora, la capa final de nuestra red tiene solo una salida. Esta nueva capa ha sido inicializada de manera aleatoria, por lo que esta nueva red no nos va a servir de mucho. Sin embargo, podríamos volver a entrenar esta red en otro problema en el que solo necesitemos una salida aprovechando los pesos que ya hemos entrenado anteriormente con el dataset MNIST. Esto es la base del *transfer learning*, una técnica que utilizaremos muchísimo más adelante y la cual explicaremos en detalle.

A continuación encontrarás varios trucos a la hora de crear redes neuronales a partir de otras que te pueden resultar útiles.

In [ ]:
# obtener una lista con las capas de una red

list(model.children())

[Linear(in_features=784, out_features=100, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=1, bias=True)]

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)

new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
)

In [ ]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)

new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
)

## Resumen

En este post hemos visto la funcionalidad que `Pytorch` nos ofrece a la hora de definir y entrenar nuestras `redes neuronales`. El paquete `torch.nn` contiene todo lo necesario para diseñar nuestros modelos, ya sea de manera secuencial o con una clase *custom* para arquitecturas más complicadas. También nos da muchas funciones de pérdida que podemos usar directamente para entrenar las redes. Te recomiendo encarecidamente que le eches un vistazo a la [documentación](https://pytorch.org/docs/stable/nn.html) par hacerte una idea de todo lo que puedes hacer. También hemos visto como el paquete `torch.optim` nos oferece algoritmos de optimización que también nos hacen la vida más fácil a la hora de entrenar nuestras redes.